In [1]:
import requests 
from secret import key
import pandas as pd

from collections import defaultdict

In [2]:
abbrev_file = 'us-state-abbreviations.txt'
with open(abbrev_file, 'r') as file:
    # Read the file contents and split by lines
    state_abbreviations = [line.strip() for line in file.readlines()]

# Print the list of state abbreviations
print(state_abbreviations)

['AK', 'AL', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY']


In [85]:
def get_st_df(state, student_size=1000):
    # Base URL for the College Scorecard API
    url = "https://api.data.gov/ed/collegescorecard/v1/schools"
    
    # Your API key (replace with your own key)
    api_key = key
    
    fields = ['school.name',
              'latest.student.size',
              'school.state',
              'latest.admissions.admission_rate.overall',
              'latest.admissions.sat_scores.average.overall',
              'latest.admissions.act_scores.midpoint.cumulative',
              'latest.admissions.test_requirements',
              f'student.size__range={5000}..']
    
    params = {
        'api_key': api_key,
        'school.state': state, 
        'fields': ','.join(fields), 
        'page': 0,  # Page number for pagination

        # ---- NEEED TO INCREASE THIS NUMBER!
        'per_page': 50  # Number of records per page (you can adjust this)
    }
    
    # Send the GET request
    response = requests.get(url, params=params)
    state_data = response.json()['results']

    return state_data 

In [86]:
# get_st_df('AL')

In [87]:
# try to get a call and iterate thorugh each state and store in a single dataframe! 
def get_all_states_data(key, st_list):
    """ Iterates through all states and collects the data for each
    Params:
    - key = API key
    Returns:
    A dictionary with states as keys and corresponding school data as values """
    
    # Dictionary to store data for all states
    all_states_data = []
    
    # Loop through all states
    for state in st_list:
        print(f"Retrieving data for {state}...")
        state_data = get_st_df(state)
        
        all_states_data.append(state_data)
        
    # # Combine all state data into a single DataFrame
    # combined_df = pd.concat(all_states_data, ignore_index=True)
    return all_states_data

In [88]:
all_states = get_all_states_data(key, state_abbreviations)

# # first test with the first 6 states 
# test_st = state_abbreviations[:2]

# all_states = get_all_states_data(key, test_st)

Retrieving data for AK...
Retrieving data for AL...
Retrieving data for AZ...
Retrieving data for AR...
Retrieving data for CA...
Retrieving data for CO...
Retrieving data for CT...
Retrieving data for DE...
Retrieving data for FL...
Retrieving data for GA...
Retrieving data for HI...
Retrieving data for ID...
Retrieving data for IL...
Retrieving data for IN...
Retrieving data for IA...
Retrieving data for KS...
Retrieving data for KY...
Retrieving data for LA...
Retrieving data for ME...
Retrieving data for MD...
Retrieving data for MA...
Retrieving data for MI...
Retrieving data for MN...
Retrieving data for MS...
Retrieving data for MO...
Retrieving data for MT...
Retrieving data for NE...
Retrieving data for NV...
Retrieving data for NH...
Retrieving data for NJ...
Retrieving data for NM...
Retrieving data for NY...
Retrieving data for NC...
Retrieving data for ND...
Retrieving data for OH...
Retrieving data for OK...
Retrieving data for OR...
Retrieving data for PA...
Retrieving d

In [89]:
flattened_data = [school for state_data in all_states for school in state_data]

# Convert to a DataFrame
df = pd.DataFrame(flattened_data)

df

,latest.student.size,latest.admissions.admission_rate.overall,latest.admissions.sat_scores.average.overall,latest.admissions.act_scores.midpoint.cumulative,latest.admissions.test_requirements,school.name,school.state
0,6778,0.6532,NaN,NaN,3,University of Alaska Anchorage,AK
1,31,NaN,NaN,NaN,0,Alaska Bible College,AK
2,4010,NaN,NaN,22.0,0,University of Alaska Fairbanks,AK
3,964,0.6266,NaN,NaN,3,University of Alaska Southeast,AK
4,457,0.9931,NaN,NaN,3,Alaska Pacific University,AK
...,...,...,...,...,...,...,...
2156,1700,NaN,NaN,NaN,0,Northern Wyoming Community College District,WY
2157,1239,NaN,NaN,NaN,0,Western Wyoming Community College,WY
2158,45,NaN,NaN,NaN,0,Cheeks Beauty Academy,WY
2159,993,NaN,NaN,NaN,0,WyoTech,WY


In [90]:
# --- convert numerical requirements to strings 
test_labels = {
        0: 'Not Required',
        1: 'Required',
        2: 'Recommended',
        3: 'Niether Rec. or Req.',
        4: 'Not Known'}

df['test_requirement_label'] = df['latest.admissions.test_requirements'].map(
    lambda x: test_labels.get(x, 'Considered but not Req.')
)

df.head()

,latest.student.size,latest.admissions.admission_rate.overall,latest.admissions.sat_scores.average.overall,latest.admissions.act_scores.midpoint.cumulative,latest.admissions.test_requirements,school.name,school.state,test_requirement_label
0,6778,0.6532,NaN,NaN,3,University of Alaska Anchorage,AK,Niether Rec. or Req.
1,31,NaN,NaN,NaN,0,Alaska Bible College,AK,Not Required
2,4010,NaN,NaN,22.0,0,University of Alaska Fairbanks,AK,Not Required
3,964,0.6266,NaN,NaN,3,University of Alaska Southeast,AK,Niether Rec. or Req.
4,457,0.9931,NaN,NaN,3,Alaska Pacific University,AK,Niether Rec. or Req.


In [91]:
nan_count = df.isna().sum()
nan_count

latest.student.size                                    0
latest.admissions.admission_rate.overall            1295
latest.admissions.sat_scores.average.overall        1595
latest.admissions.act_scores.midpoint.cumulative    1635
latest.admissions.test_requirements                    0
school.name                                            0
school.state                                           0
test_requirement_label                                 0
dtype: int64

In [92]:
df[['latest.admissions.admission_rate.overall', 'latest.admissions.sat_scores.average.overall', 'latest.admissions.act_scores.midpoint.cumulative', 'school.state']].groupby(by='school.state').mean()


,latest.admissions.admission_rate.overall,latest.admissions.sat_scores.average.overall,latest.admissions.act_scores.midpoint.cumulative
school.state,,,
AK,0.757633,NaN,22.000000
AL,0.760400,1133.375000,22.312500
AR,0.712137,1101.714286,22.833333
AZ,0.827987,1253.500000,23.666667
CA,0.632703,1213.333333,24.666667
CO,0.774490,1198.000000,25.076923
CT,0.637850,1257.933333,29.200000
DE,0.621300,1114.500000,29.000000
FL,0.680262,1158.750000,24.066667


In [93]:
df.shape

(2161, 8)